### Lets start the cleaning process by importing the necessary libraries and setting some default visual parameters for better viewing

In [1]:
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import os
from pathlib import Path

pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)


In [2]:
# path to the raw wikidata CSV
file_path = "/workspaces/dsi-ws2025-project-grpab-weigl-mds1ab-awp-proj2/Data_Acquisition/wikidata_eu_companies_raw.csv"  

df = pd.read_csv(file_path, dtype=str).fillna("")
print("Loaded shape:", df.shape)
df.head(5)


Loaded shape: (174711, 12)


,item,inception,website,itemLabel,industryLabel,hqLabel,ownedByLabel,memberOfLabel,_country_q,dissolved,revenue,revenue_date
0,http://www.wikidata.org/entity/Q83822,1919-11-21T00:00:00Z,https://konzern.oebb.at/,Austrian Federal Railways,rail freight transport,ÖBB Konzernzentrale Hauptbahnhof Wien,Austria,International Union of Railways,wd:Q40,,,
1,http://www.wikidata.org/entity/Q83822,2004-03-31T00:00:00Z,https://konzern.oebb.at/,Austrian Federal Railways,rail freight transport,ÖBB Konzernzentrale Hauptbahnhof Wien,Austria,International Union of Railways,wd:Q40,,,
2,http://www.wikidata.org/entity/Q83822,1919-11-21T00:00:00Z,https://konzern.oebb.at/,Austrian Federal Railways,rail transport,ÖBB Konzernzentrale Hauptbahnhof Wien,Austria,International Union of Railways,wd:Q40,,,
3,http://www.wikidata.org/entity/Q83822,2004-03-31T00:00:00Z,https://konzern.oebb.at/,Austrian Federal Railways,rail transport,ÖBB Konzernzentrale Hauptbahnhof Wien,Austria,International Union of Railways,wd:Q40,,,
4,http://www.wikidata.org/entity/Q83822,1919-11-21T00:00:00Z,https://konzern.oebb.at/,Austrian Federal Railways,"passenger rail transport, interurban (NACE 49.1)",ÖBB Konzernzentrale Hauptbahnhof Wien,Austria,International Union of Railways,wd:Q40,,,


##### Now that the data is loaded, let us first try to do some basic tidying operations like trimming spaces, unifying NaNs into a single recognisable value as well as taking a quick look at the data schema.

The idea is to carry out basic cleaning steps first, understand the structure and content of data to carry out specific cleaning and modifying steps if necessary

In [3]:
# stripping whitespace from all string cells
df = df.map(lambda x: x.strip() if isinstance(x, str) else x)

# convert "" to NaN for proper null handling
df = df.replace({"": np.nan})

print("Columns:", list(df.columns))
print("Shape after initial tidy:", df.shape)
print("\nNull counts per column:\n", df.isna().sum())


Columns: ['item', 'inception', 'website', 'itemLabel', 'industryLabel', 'hqLabel', 'ownedByLabel', 'memberOfLabel', '_country_q', 'dissolved', 'revenue', 'revenue_date']
Shape after initial tidy: (174711, 12)

Null counts per column:
 item                  0
inception         64311
website           68779
itemLabel             0
industryLabel     95889
hqLabel           74648
ownedByLabel     114707
memberOfLabel    126247
_country_q            0
dissolved        162759
revenue          121405
revenue_date     121591
dtype: int64


In [4]:
# Carrying a quick duplicate check to see if any records are duplicated
dup_count = df.duplicated().sum()
print("Exact duplicate rows:", dup_count)

Exact duplicate rows: 44948


We find a surprisingly huge number of duplicated rows possibly due to the fact that we did not scrape the complete information of companies from wikidata, just the necessary ones. So its easy to assume multiple records for companies. We will come back to this result later to handle it

In [5]:
for col in ["inception", "dissolved", "revenue_date"]:
    if col in df.columns:
        df[col + "_parsed"] = pd.to_datetime(df[col], errors="coerce")

df[["inception", "inception_parsed", "dissolved", "dissolved_parsed", "revenue", "revenue_date", "revenue_date_parsed"]].head(10)


,inception,inception_parsed,dissolved,dissolved_parsed,revenue,revenue_date,revenue_date_parsed
0,1919-11-21T00:00:00Z,1919-11-21 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
1,2004-03-31T00:00:00Z,2004-03-31 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
2,1919-11-21T00:00:00Z,1919-11-21 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
3,2004-03-31T00:00:00Z,2004-03-31 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
4,1919-11-21T00:00:00Z,1919-11-21 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
5,2004-03-31T00:00:00Z,2004-03-31 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
6,1919-11-21T00:00:00Z,1919-11-21 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
7,2004-03-31T00:00:00Z,2004-03-31 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
8,1919-11-21T00:00:00Z,1919-11-21 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT
9,2004-03-31T00:00:00Z,2004-03-31 00:00:00+00:00,NaN,NaT,NaN,NaN,NaT


Here, I just checked the usual datetime columns to understand any inconsistencies in it and creating additional columns to see if the coercion worked. As we can see, a lot of these columns are just nulls so we will come back later to deal with these

In [6]:
def extract_domain(url):
    if pd.isna(url): 
        return np.nan
    try:
        netloc = urlparse(url).netloc.lower()
        if netloc.startswith("www."):
            netloc = netloc[4:]
        return netloc or np.nan
    except Exception:
        return np.nan

if "website" in df.columns:
    df["website_domain"] = df["website"].apply(extract_domain)

df[["website", "website_domain"]].head(10)

,website,website_domain
0,https://konzern.oebb.at/,konzern.oebb.at
1,https://konzern.oebb.at/,konzern.oebb.at
2,https://konzern.oebb.at/,konzern.oebb.at
3,https://konzern.oebb.at/,konzern.oebb.at
4,https://konzern.oebb.at/,konzern.oebb.at
5,https://konzern.oebb.at/,konzern.oebb.at
6,https://www.oebb.at/,oebb.at
7,https://www.oebb.at/,oebb.at
8,https://www.oebb.at/,oebb.at
9,https://www.oebb.at/,oebb.at


This step is to trim down and normalize website related columns to preview and see if the data here is formatted properly or not. Looks like its good to go looking at the preview

In [7]:
# Checking to see the number of records and unique values per item/company?
grp = df.groupby("item").agg(
    rows=("item","size"),
    n_inception=("inception_parsed","nunique"),
    n_website=("website_domain","nunique"),
    n_industry=("industryLabel","nunique"),
    n_hq=("hqLabel","nunique"),
    n_memberOf=("memberOfLabel","nunique"),
    n_revenue_entries=("revenue","nunique")
).reset_index()

print("Items summary shape:", grp.shape)
grp.sort_values("rows", ascending=False).head(10)


Items summary shape: (89052, 8)


,item,rows,n_inception,n_website,n_industry,n_hq,n_memberOf,n_revenue_entries
87386,http://www.wikidata.org/entity/Q9396,17650,1,1,2,1,52,32
53450,http://www.wikidata.org/entity/Q2311,6878,1,1,4,1,4,29
81270,http://www.wikidata.org/entity/Q661845,4967,1,1,1,1,16,25
87283,http://www.wikidata.org/entity/Q9322,3062,2,1,3,1,24,6
69577,http://www.wikidata.org/entity/Q334204,2895,4,1,2,1,3,15
29959,http://www.wikidata.org/entity/Q1172038,900,1,1,3,1,1,25
45546,http://www.wikidata.org/entity/Q1550912,829,1,1,6,1,0,21
60476,http://www.wikidata.org/entity/Q26794476,673,2,2,1,0,6,11
76187,http://www.wikidata.org/entity/Q52825,606,1,1,2,2,9,2
28810,http://www.wikidata.org/entity/Q1161666,584,1,1,6,2,4,13


We can see the number of items as around 89000

Lets now zero down on one particular example (picking the one with most duplicates in this case) to understand the data, its structure and what to standardize in it

In [8]:
top_item = grp.sort_values("rows", ascending=False).iloc[0]["item"]
print("Top item with most rows:", top_item)
print("Number of rows for this top item:", len(df[df["item"] == top_item]))

df[df["item"] == top_item].head(20)

Top item with most rows: http://www.wikidata.org/entity/Q9396
Number of rows for this top item: 17650


,item,inception,website,itemLabel,industryLabel,hqLabel,ownedByLabel,memberOfLabel,_country_q,dissolved,revenue,revenue_date,inception_parsed,dissolved_parsed,revenue_date_parsed,website_domain
11476,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,telecommunications,Bonn,Germany,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11477,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,mobile phone industry,Bonn,Germany,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11478,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,telecommunications,Bonn,SoftBank Group,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11479,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,mobile phone industry,Bonn,SoftBank Group,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11480,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,telecommunications,Bonn,BlackRock,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11481,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,mobile phone industry,Bonn,BlackRock,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11482,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,telecommunications,Bonn,KfW,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11483,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,mobile phone industry,Bonn,KfW,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11484,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,telecommunications,Bonn,Government Pension Fund Global,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com
11485,http://www.wikidata.org/entity/Q9396,1995-01-01T00:00:00Z,https://telekom.com/,Deutsche Telekom,mobile phone industry,Bonn,Government Pension Fund Global,World Wide Web Consortium,wd:Q183,NaN,115769000000,2024-12-31T00:00:00Z,1995-01-01 00:00:00+00:00,NaT,2024-12-31 00:00:00+00:00,telekom.com


As we can see, Deutsche Telekom has the most number of rows with a whopping 17650 rows as we display the first 20 rows. We can see that the industrylabel has multiple values along with the field for ownedBy. Now that we have a clear snapshot of the kind of data we're dealing with, let us now try and clean this dataset

We will start with checking how much data is exactly duplicated and figure out the right way to clean it up

In [9]:
# Dropping fully identical rows
before = df.shape[0]
df = df.drop_duplicates()
after = df.shape[0]

print(f"Exact duplicates dropped: {before - after}")
print(f"Remaining rows: {after}")

# Also checking for 'quasi-duplicates' (same item + website + industry)
dup_cols = ['item', 'website', 'industryLabel']
dupes = df[df.duplicated(subset=dup_cols, keep=False)].sort_values(dup_cols)
print("Quasi-duplicate groups:", dupes.groupby('item').size().head())
dupes.head(10)


Exact duplicates dropped: 44948
Remaining rows: 129763
Quasi-duplicate groups: item
http://www.wikidata.org/entity/Q100146248    4
http://www.wikidata.org/entity/Q1003155      2
http://www.wikidata.org/entity/Q100604548    2
http://www.wikidata.org/entity/Q100712425    2
http://www.wikidata.org/entity/Q100991019    4
dtype: int64


,item,inception,website,itemLabel,industryLabel,hqLabel,ownedByLabel,memberOfLabel,_country_q,dissolved,revenue,revenue_date,inception_parsed,dissolved_parsed,revenue_date_parsed,website_domain
82757,http://www.wikidata.org/entity/Q100146248,1888-01-01T00:00:00Z,NaN,Dutch Bank for South Africa,"financial service activities, except insurance...",Amsterdam,NaN,NaN,wd:Q55,NaN,NaN,NaN,1888-01-01 00:00:00+00:00,NaT,NaT,NaN
82759,http://www.wikidata.org/entity/Q100146248,1888-01-01T00:00:00Z,NaN,Dutch Bank for South Africa,"financial service activities, except insurance...",Pretoria,NaN,NaN,wd:Q55,NaN,NaN,NaN,1888-01-01 00:00:00+00:00,NaT,NaT,NaN
82756,http://www.wikidata.org/entity/Q100146248,1888-01-01T00:00:00Z,NaN,Dutch Bank for South Africa,financial services,Amsterdam,NaN,NaN,wd:Q55,NaN,NaN,NaN,1888-01-01 00:00:00+00:00,NaT,NaT,NaN
82758,http://www.wikidata.org/entity/Q100146248,1888-01-01T00:00:00Z,NaN,Dutch Bank for South Africa,financial services,Pretoria,NaN,NaN,wd:Q55,NaN,NaN,NaN,1888-01-01 00:00:00+00:00,NaT,NaT,NaN
59043,http://www.wikidata.org/entity/Q1003155,1827-01-01T00:00:00Z,https://www.buitoni.it,Buitoni,NaN,Milan,NewPrinces,NaN,wd:Q38,NaN,NaN,NaN,1827-01-01 00:00:00+00:00,NaT,NaT,buitoni.it
59044,http://www.wikidata.org/entity/Q1003155,1827-01-01T00:00:00Z,https://www.buitoni.it,Buitoni,NaN,Sansepolcro,NewPrinces,NaN,wd:Q38,NaN,NaN,NaN,1827-01-01 00:00:00+00:00,NaT,NaT,buitoni.it
35718,http://www.wikidata.org/entity/Q100604548,1981-01-01T00:00:00Z,http://www.rpbw.com/,Renzo Piano Building Workshop,NaN,NaN,NaN,NaN,wd:Q142,NaN,NaN,NaN,1981-01-01 00:00:00+00:00,NaT,NaT,rpbw.com
53581,http://www.wikidata.org/entity/Q100604548,1981-01-01T00:00:00Z,http://www.rpbw.com/,Renzo Piano Building Workshop,NaN,NaN,NaN,NaN,wd:Q38,NaN,NaN,NaN,1981-01-01 00:00:00+00:00,NaT,NaT,rpbw.com
142665,http://www.wikidata.org/entity/Q100712425,1860-01-01T00:00:00Z,NaN,Fretz Brothers Ltd.,NaN,Zurich,NaN,Q100782934,wd:Q39,1980-01-01T00:00:00Z,NaN,NaN,1860-01-01 00:00:00+00:00,1980-01-01 00:00:00+00:00,NaT,NaN
142666,http://www.wikidata.org/entity/Q100712425,1914-01-01T00:00:00Z,NaN,Fretz Brothers Ltd.,NaN,Zurich,NaN,Q100782934,wd:Q39,1980-01-01T00:00:00Z,NaN,NaN,1914-01-01 00:00:00+00:00,1980-01-01 00:00:00+00:00,NaT,NaN


We have now successfully cleared the exact duplicates and also checked for the so-called quasi or semi duplicates. These records only differ by minor factors like websites, labels, HQs, owners etc. which we will try and group to handle them now.

In [10]:
# Specifying the textual columns that differ for same companies
text_cols = [
    "itemLabel", "industryLabel", "hqLabel",
    "ownedByLabel", "memberOfLabel", "website"
]

#Trimming and converting websites to lowercase websites
df["website"] = (
    df["website"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"nan": np.nan, "": np.nan})
)

#Removing the trailing slashes and 'http[s]://www.' prefixes for further cleaning
df["website"] = (
    df["website"]
    .str.replace(r"^https?://(www\.)?", "", regex=True)
    .str.rstrip("/")
)

#Standardizing the text casing and trimming for other columns
for c in text_cols:
    if c == "website":
        continue
    df[c] = (
        df[c]
        .astype(str)
        .str.strip()
        .replace({"nan": np.nan, "": np.nan})
    )

#Simplifying the 'industryLabel' and 'hqLabel' to title case for better consistency
df["industryLabel"] = df["industryLabel"].str.title()
df["hqLabel"] = df["hqLabel"].str.title()

# Previewing the unique website formats and potential anomalies before we proceed further
df["website"].head(10), df["website"].nunique()


(0    konzern.oebb.at
 1    konzern.oebb.at
 2    konzern.oebb.at
 3    konzern.oebb.at
 4    konzern.oebb.at
 5    konzern.oebb.at
 6            oebb.at
 7            oebb.at
 8            oebb.at
 9            oebb.at
 Name: website, dtype: object,
 34069)

As we can see, the domains are now correctly formatted without the messy prefixes behind them. And we also see around 34000 unique domains across europe which is again reasonable given our dataset size and scope.

Now, let us go beyond and try to consolidate each company into a single unified record while trying to retain the most important of its attributes

In [11]:
def first_valid(series):
    return series.dropna().iloc[0] if series.notna().any() else np.nan

def unique_list(series):
    vals = series.dropna().unique().tolist()
    return ", ".join(sorted(vals)) if vals else np.nan

# Converting revenue_date to datetime if not already parsed
df["revenue_date_parsed"] = pd.to_datetime(df["revenue_date"], errors="coerce")

# Grouping by Wikidata entity for consolidation
agg_dict = {
    "itemLabel": first_valid,
    "_country_q": first_valid,
    "website": first_valid,
    "industryLabel": unique_list,
    "hqLabel": unique_list,
    "ownedByLabel": unique_list,
    "memberOfLabel": unique_list,
    "inception": first_valid,
    "dissolved": first_valid,
}

# For revenue, we decided to choose latest entry per company
revenue_df = (
    df.sort_values("revenue_date_parsed", ascending=False)
    .drop_duplicates(subset=["item"], keep="first")[["item", "revenue", "revenue_date_parsed"]]
)

# Aggregating everything we've done so far for all companies
df_cleaned = df.groupby("item").agg(agg_dict).reset_index()

# Merging back the latest revenue info
df_cleaned = df_cleaned.merge(revenue_df, on="item", how="left")

print("Unique companies after consolidation:", len(df_cleaned))
df_cleaned.head(10)


Unique companies after consolidation: 89052


,item,itemLabel,_country_q,website,industryLabel,hqLabel,ownedByLabel,memberOfLabel,inception,dissolved,revenue,revenue_date_parsed
0,http://www.wikidata.org/entity/Q1000428,MySQL AB,wd:Q34,mysql.com,Software Industry,Solna Municipality,Oracle America,NaN,2001-01-01T00:00:00Z,NaN,NaN,NaT
1,http://www.wikidata.org/entity/Q100045703,Samaki Wanne,wd:Q29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,http://www.wikidata.org/entity/Q1000751,Ferrovie del Gargano,wd:Q38,ferroviedelgargano.com,NaN,Bari,NaN,NaN,1962-01-01T00:00:00Z,NaN,NaN,NaT
3,http://www.wikidata.org/entity/Q1000752,Ferrovie della Calabria,wd:Q38,ferroviedellacalabria.it,Public Transport,Catanzaro,NaN,NaN,2001-01-01T00:00:00Z,NaN,NaN,NaT
4,http://www.wikidata.org/entity/Q100089670,CanalSat Suisse,wd:Q39,canalsat.ch,NaN,Lausanne,NaN,NaN,NaN,NaN,NaN,NaT
5,http://www.wikidata.org/entity/Q1000909,Tiefrastenhütte,wd:Q38,schutzhuetten.net/tiefrastenhuette.html,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
6,http://www.wikidata.org/entity/Q100094013,Telia Danmark,wd:Q35,telia.dk,Telecommunications,Copenhagen,Norlys Energi,NaN,1995-05-01T00:00:00Z,NaN,NaN,NaT
7,http://www.wikidata.org/entity/Q1000943,Sophya,wd:Q55,sophya.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8,http://www.wikidata.org/entity/Q100108734,Q100108734,wd:Q55,pro-facto.nl,NaN,Groningen,NaN,NaN,NaN,NaN,NaN,NaT
9,http://www.wikidata.org/entity/Q100135579,Q100135579,wd:Q40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


The idea is that companies appear multiple times due to differences in industry phrasing, multiple HQs or owners and some repeating revenue snapshots. Above, we are trying to group the dataset by item as we take the first non-null value  for stable fields, aggregating the categorical fields and picking the latest revenue entry if multiple exist.

As initially expected, we get around 89000 unique companies after consolidation which is already a lot for our project.

Next up, let us now proceed ahead with handling the data that includes cleaning, standardizing etc. To start, let us first standardize the country codes and convert date fields

In [12]:
# Mapping Wikidata country QIDs to readable names
country_map = {
    "wd:Q40": "Austria", "wd:Q183": "Germany", "wd:Q142": "France",
    "wd:Q38": "Italy", "wd:Q55": "Netherlands", "wd:Q29": "Spain",
    "wd:Q34": "Sweden", "wd:Q145": "United Kingdom", "wd:Q39": "Switzerland",
    "wd:Q20": "Norway", "wd:Q35": "Denmark"
}
df_cleaned["country"] = df_cleaned["_country_q"].map(country_map)
df_cleaned.drop(columns=["_country_q"], inplace=True)

# Parsing dates for proper formatting
for col in ["inception", "dissolved", "revenue_date_parsed"]:
    df_cleaned[col] = pd.to_datetime(df_cleaned[col], errors="coerce")

# Cleaning numeric revenue
df_cleaned["revenue"] = pd.to_numeric(df_cleaned["revenue"], errors="coerce")

# Getting and checking the summary of missing counts
print("Null counts after conversions:")
print(df_cleaned.isna().sum().sort_values(ascending=False).head(10))

df_cleaned.head(5)


Null counts after conversions:
revenue_date_parsed    88311
revenue                88267
memberOfLabel          87755
ownedByLabel           83014
dissolved              81616
industryLabel          76366
hqLabel                60347
website                55870
inception              53962
itemLabel                  0
dtype: int64


,item,itemLabel,website,industryLabel,hqLabel,ownedByLabel,memberOfLabel,inception,dissolved,revenue,revenue_date_parsed,country
0,http://www.wikidata.org/entity/Q1000428,MySQL AB,mysql.com,Software Industry,Solna Municipality,Oracle America,NaN,2001-01-01 00:00:00+00:00,NaT,NaN,NaT,Sweden
1,http://www.wikidata.org/entity/Q100045703,Samaki Wanne,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaT,Spain
2,http://www.wikidata.org/entity/Q1000751,Ferrovie del Gargano,ferroviedelgargano.com,NaN,Bari,NaN,NaN,1962-01-01 00:00:00+00:00,NaT,NaN,NaT,Italy
3,http://www.wikidata.org/entity/Q1000752,Ferrovie della Calabria,ferroviedellacalabria.it,Public Transport,Catanzaro,NaN,NaN,2001-01-01 00:00:00+00:00,NaT,NaN,NaT,Italy
4,http://www.wikidata.org/entity/Q100089670,CanalSat Suisse,canalsat.ch,NaN,Lausanne,NaN,NaN,NaT,NaT,NaN,NaT,Switzerland


As we can see, there are a lot of null values for each field in this dataset probably due to the unavailability of data, especially the revenue ones. We have to make a decision on what to do with this data as filling it would either require manual intervention or usage of GPT to iterate through each row both of which would be beyond the effort scope for this project

Strategy for cleaning:

Revenue exists for <1% of companies, inception has around 60% coverage while website has around 40% coverage. Keeping all the 89k companies would not make sense in this case as the data completeness for them is really poor. This would ultimately do more harm than good considering both the analysis angle and the storage space angle.

So, let us now try to define our pwn filter criteria to filter out companies and get the ones with complete/near complete data. To filter out only modern companies/startups, we will assume

1. Companies founded after 2000
2. Website and industry space not null
3. Excluding entities like banks, municipalities, government agencies, political entities so we dont skew the analysis
4. Making sure the list of filtered countries must be commercial economies for richer data

Lets begin by assessing how many records have no meaningful business info. We are cusotm defining business info as info on either website or hqLabel or industryLabel or inception

In [15]:
df_cleaned["has_core_info"] = (
    df_cleaned["website"].notna() |
    df_cleaned["hqLabel"].notna() |
    df_cleaned["industryLabel"].notna() |
    df_cleaned["inception"].notna()
)

df_cleaned["has_country"] = df_cleaned["country"].notna()

df_cleaned["keep_row"] = df_cleaned["has_core_info"] & df_cleaned["has_country"]
print(df_cleaned[df_cleaned['keep_row']==True].shape)
df_cleaned[["has_core_info", "has_country", "keep_row"]].head()


(53328, 15)


,has_core_info,has_country,keep_row
0,True,True,True
1,False,True,False
2,True,True,True
3,True,True,True
4,True,True,True


In [16]:
# Dropping the rows with no info
before = df_cleaned.shape[0]
df_cleaned = df_cleaned[df_cleaned["keep_row"]].drop(columns=["has_core_info", "has_country", "keep_row"])
after = df_cleaned.shape[0]

print(f"Rows removed due to insufficient information: {before - after}")
print(f"Remaining rows: {after}")


Rows removed due to insufficient information: 35724
Remaining rows: 53328


Filtering out the data even further as 50k records are still quite a lot and we can afford to be a bit more strict in data quality as long we can get more than 2-5k companies

In [17]:
df_cleaned = df_cleaned[
    df_cleaned["website"].notna() &
    df_cleaned["industryLabel"].notna() &
    df_cleaned["inception"].notna() &
    df_cleaned["hqLabel"].notna() &
    df_cleaned["country"].notna()
].copy()

print("Rows after strict filter:", df_cleaned.shape)
df_cleaned.head()


Rows after strict filter: (6384, 12)


,item,itemLabel,website,industryLabel,hqLabel,ownedByLabel,memberOfLabel,inception,dissolved,revenue,revenue_date_parsed,country
0,http://www.wikidata.org/entity/Q1000428,MySQL AB,mysql.com,Software Industry,Solna Municipality,Oracle America,NaN,2001-01-01 00:00:00+00:00,NaT,NaN,NaT,Sweden
3,http://www.wikidata.org/entity/Q1000752,Ferrovie della Calabria,ferroviedellacalabria.it,Public Transport,Catanzaro,NaN,NaN,2001-01-01 00:00:00+00:00,NaT,NaN,NaT,Italy
6,http://www.wikidata.org/entity/Q100094013,Telia Danmark,telia.dk,Telecommunications,Copenhagen,Norlys Energi,NaN,1995-05-01 00:00:00+00:00,NaT,NaN,NaT,Denmark
11,http://www.wikidata.org/entity/Q100142778,Brownies&downieS,browniesanddownies.nl,Horeca,Veghel,NaN,NaN,2010-01-01 00:00:00+00:00,NaT,NaN,NaT,Netherlands
28,http://www.wikidata.org/entity/Q100166679,librerie.coop,librerie.coop,Book Retail Industry,Villanova,Coop Alleanza 3.0,NaN,2006-01-01 00:00:00+00:00,NaT,NaN,NaT,Italy


We finally get data with the shape of 6384 companies where most of the core information exists. We can now do the final cleaning on this subset to create the final dataset for this source

In [18]:
df_cleaned.isna().sum().sort_values(ascending=False)

memberOfLabel          5930
dissolved              5856
revenue_date_parsed    5848
revenue                5819
ownedByLabel           5013
website                   0
itemLabel                 0
item                      0
industryLabel             0
hqLabel                   0
inception                 0
country                   0
dtype: int64

In [19]:
(df_cleaned.isna().mean() * 100).round(2)

item                    0.00
itemLabel               0.00
website                 0.00
industryLabel           0.00
hqLabel                 0.00
ownedByLabel           78.52
memberOfLabel          92.89
inception               0.00
dissolved              91.73
revenue                91.15
revenue_date_parsed    91.60
country                 0.00
dtype: float64

The revenue information is mostly missing in wikidata anyway so we can leave it as it is for now. While the dissolved column is interesting and important, we can also assume that the companies might not be dissolved which could also result in null values.

Seeing as to the missing data easily constitutes more than 80% for these features which are not exactly core features, we can still keep their features

In [21]:
# We add unknown categories wherever applicable and ship it as the cleaned dataset. We still have nulls but the data is mostly clean and analysis ready

df_imputed = df_cleaned.copy()
df_imputed["industryLabel"] = df_imputed["industryLabel"].fillna("Unknown Industry")
df_imputed["hqLabel"] = df_imputed["hqLabel"].fillna("Unknown HQ")
len(df_imputed)


6384

In [22]:
# Saving the cleaned file as a csv for further use and ending the notebook here
output_path = "wikidata_clean_eu_companies.csv"

df_cleaned.to_csv(output_path, index=False, encoding="utf-8")

print(f"Exported cleaned dataset to: {output_path}")
print(df_cleaned.shape)
df_cleaned.head()


Exported cleaned dataset to: wikidata_clean_eu_companies.csv
(6384, 12)


,item,itemLabel,website,industryLabel,hqLabel,ownedByLabel,memberOfLabel,inception,dissolved,revenue,revenue_date_parsed,country
0,http://www.wikidata.org/entity/Q1000428,MySQL AB,mysql.com,Software Industry,Solna Municipality,Oracle America,NaN,2001-01-01 00:00:00+00:00,NaT,NaN,NaT,Sweden
3,http://www.wikidata.org/entity/Q1000752,Ferrovie della Calabria,ferroviedellacalabria.it,Public Transport,Catanzaro,NaN,NaN,2001-01-01 00:00:00+00:00,NaT,NaN,NaT,Italy
6,http://www.wikidata.org/entity/Q100094013,Telia Danmark,telia.dk,Telecommunications,Copenhagen,Norlys Energi,NaN,1995-05-01 00:00:00+00:00,NaT,NaN,NaT,Denmark
11,http://www.wikidata.org/entity/Q100142778,Brownies&downieS,browniesanddownies.nl,Horeca,Veghel,NaN,NaN,2010-01-01 00:00:00+00:00,NaT,NaN,NaT,Netherlands
28,http://www.wikidata.org/entity/Q100166679,librerie.coop,librerie.coop,Book Retail Industry,Villanova,Coop Alleanza 3.0,NaN,2006-01-01 00:00:00+00:00,NaT,NaN,NaT,Italy
